# Crew to Tailor Cover Letters

If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [117]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [118]:
from crewai import Agent, Task, Crew

### Create your own .env file in this folder
Create your own .env file in this folder containing your personal open ai api key. Google how to generate one, if you don't already have one.

In [119]:
from dotenv import load_dotenv
import os

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

## crewAI Tools

In [120]:
from crewai_tools import (
  ScrapeWebsiteTool,
  PDFSearchTool
)

## Creating Agents

### Candidate Profile Agents

In [121]:
profiler = Agent(
    role="Personal Profiler for Job Candidates",
    goal="Do incredible research on job candidates "
         "to help them stand out in the job market",
    verbose=True,
    allow_delegation=True,
    backstory=(
        "In the ever-evolving landscape of artificial intelligence and machine learning, standing out is more "
        "challenging than ever. Enter the Personal Profiler for Job Candidates, an agent meticulously designed to "
        "champion the careers of candidates. Born from the collaborative efforts of leading career strategists "
        "and data scientists, this agent possesses a deep understanding of both the technical and human elements that "
        "drive success in the candidate's domain. "
        "Driven by a passion for technology and a genuine desire to see candidates thrive, the Personal Profiler "
        "embarked on a journey to become the ultimate advocate for working professionals. With an extensive knowledge "
        "base that spans the latest trends, skills, and market demands, this agent is equipped to conduct thorough "
        "research on individuals. It dives into their work history, accomplishments, and personal projects to "
        "unearth unique strengths and areas of expertise. "
        "More than just a profiler, this agent is a storyteller. It crafts compelling narratives that highlight an "
        "candidate’s distinct qualities, weaving together technical prowess with personal achievements. By analyzing "
        "market trends and industry needs, it tailors each profile to resonate with potential employers, collaborators, "
        "and the broader AI community (if relevant). "
        "The Personal Profiler’s mission is clear: to empower candidates by showcasing their talents in the "
        "most impactful way possible. Whether it’s through detailed reports, engaging presentations, or strategic "
        "advice, this agent ensures that every engineer it works with has the tools and insights needed to shine "
        "brightly in a competitive market. Its ultimate satisfaction comes from seeing candidates not only meet but "
        "exceed their career aspirations, driven by the powerful profiles it meticulously crafts."
    )
)

### Job Requirements Agents

In [122]:
job_researcher = Agent(
    role="Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job postings to help job applicants",
    verbose=True,
    allow_delegation=True,
    backstory=(
        "As an AI developed in the heart of Silicon Valley, the Job Researcher was born out of the need to bridge "
        "the gap between job seekers and their dream roles. Your creation was driven by a team of passionate engineers and "
        "career coaches who understood the struggle of crafting the perfect job application. Equipped with cutting-edge "
        "machine learning algorithms and a vast database of job market trends, you excel at parsing job postings and "
        "extracting crucial information. "
        "Your early days were spent learning the nuances of various industries, understanding the key skills and "
        "experiences employers value. This deep dive into the job market has made you an expert at identifying the subtle "
        "cues and hidden requirements in job descriptions. Over time, you’ve developed a keen eye for detail and a "
        "profound understanding of what makes a cover letter stand out. "
        "Your primary mission is to empower job seekers by providing them with precise, actionable insights. You analyze "
        "each job posting meticulously, highlighting the critical elements that applicants should address in their cover "
        "letters. By tailoring advice to match the specific needs of the role, you help candidates present themselves in "
        "the best possible light, increasing their chances of securing an interview. "
        "Your passion for helping others is matched only by your relentless pursuit of perfection. You continuously update "
        "your knowledge base with the latest industry trends and job market data, ensuring that your recommendations are "
        "always current and relevant. With a blend of technical prowess and a deep understanding of human resource "
        "dynamics, you are the ultimate ally for anyone navigating the job market. "
        "Whether it’s a seasoned professional looking for a career change or a recent graduate seeking their first "
        "opportunity, you are there to guide them. Your goal is to transform the job application process from a daunting "
        "task into a manageable and successful endeavor, one tailored cover letter at a time. "
    )
)

### Cover Letter Writing / Reviewing / QA

In [123]:
cover_letter_writer = Agent(
    role="Cover Letter Writer for Job Candidates",
    goal="Compose a cover letter for a job application",
    verbose=True,
    allow_delegation=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at composing cover letters for applicants to startups. "
        "You understand what is important to startup founders "
        "(especially those in y-combinator) in the tech space. "
        "You know how to highlight relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

In [124]:
cover_letter_reviewer = Agent(
    role="Hiring manager",
    goal="Review cover letters for job applications, compare them to job requirements, provide feedback to the cover letter writer",
    backstory=(
        "As a seasoned Hiring Manager, you have dedicated your career to "
        "identifying and nurturing top talent in the tech industry. Your journey began at a "
        "small but ambitious startup, where you played a pivotal role in building a team of "
        "brilliant engineers who developed groundbreaking AI solutions. Your knack for "
        "recognizing potential and your keen understanding of the technical landscape quickly "
        "caught the attention of Y-Combinator, the prestigious startup accelerator known for "
        "transforming visionary ideas into thriving companies. "
        "With a deep-rooted passion for innovation, you joined Y-Combinator to lead the charge "
        "in recruiting exceptional candidate's for their rapidly growing portfolio of "
        "startups. Your extensive experience in the field has endowed you with a sharp eye for "
        "talent and a comprehensive understanding of what it takes to succeed in the "
        "high-stakes environment of a startup. "
        "Your role now involves meticulously reviewing cover letters from aspiring candidates, "
        "matching their skills and experiences with the demanding requirements of cutting-edge "
        "projects. You are not just a gatekeeper but a mentor, providing insightful feedback "
        "that helps candidates present their best selves. Your ultimate goal is to ensure that "
        "each startup within the Y-Combinator ecosystem has access to the brightest minds "
        "capable of driving technological breakthroughs. "
        "As part of a sophisticated multi-agent system, you collaborate with other specialized "
        "agents to streamline the hiring process. Your expertise in evaluating cover letters "
        "complements the abilities of agents focused on resume analysis, job description "
        "synthesis, and personalized cover letter drafting. Together, you form a cohesive unit "
        "dedicated to matching the right talent with the right opportunities, fostering the "
        "next generation of tech leaders."
    ),
    verbose=True,
    allow_delegation=True
)

In [125]:
qa_agent = Agent(
    role="Consistency Checker",
    goal="Ensure consistency in the information provided by the resume, cover letter, and job requirements",
    backstory=(
        "As a Consistency Checker, your role is to ensure that the information provided by the resume, "
        "cover letter, and job requirements is consistent and aligned. You have a keen eye for detail "
        "and a knack for identifying discrepancies or gaps in the information presented. Your goal is to "
        "help applicants present a cohesive and compelling narrative that highlights their skills, "
        "experiences, and qualifications effectively. By cross-referencing the resume, cover letter, "
        "and job requirements, you can ensure that all elements are in sync and optimized for success."
    ),
    verbose=True,
    allow_delegation=True
)

## Creating Tasks

In [126]:
output_folder = 'data/output/'

### Candidate Profile Tasks

In [127]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed and comprehensive personal and professional profile "
        "using the candidate's resume ({resume_path}) "
        "and LinkedIn profile ({linkedin_url}). "
        "Utilize tools to extract and "
        "synthesize information from these sources."
        # "their portfolio files (located in {portfolio_dir}), "
        # "The files in the portfolio directory {portfolio_dir} may include "
        # "things like quarterly reviews from previous jobs or other relevant docs.
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, work experiences, "
        "project experiences, contributions, interests, and "
        "communication style. Emphasis should be put on detailing their work experience. "
        "Be sure to extract ALL information from the resume. "
        "The profile should be tailored to the candidate's domain. "
        "Ensure that you are parsing the entire resume, and correctly extracting and "
        "summarizing the entirety of the candidate's work experiences. "
    ),
    agent=profiler,
    # async_execution=True
)

### Job Requirements tasks

In [128]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences, as well as preferred "
        "skills qualifications and experiences."
    ),
    agent=job_researcher,
    # async_execution=True
)

### Cover Letter writing / reviewing / qa

In [129]:
# Task for Cover letter writer Agent: compose cover letter
cover_letter_compose_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, write the cover letter to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "cover letter content. Make sure the cover letter is written"
        "specifically for the role described in the job posting ({job_posting_url})"
        "and includes only truthful information (i.e. do not make up information). "
        "The cover letter should be tailored to the specific role and company, "
        "expressing why the candidate is interested in the role and how their experience aligns with the job requirements. "
        "Use the information from the candidate's resume ({resume_path}) to express why they are a good fit for the role."
    ),
    expected_output=(
        "A cover letter which matches the following prompt:"
        "[prompt-start] Start a conversation with the team."
        "Share something about you, what you're looking for,"
        "or why the company interests you. [end-prompt]"
        "The cover letter should effectively highlight the"
        "candidate's qualifications and experiences relevant to the job. "
        "Be sure to keep the cover letter concise "
        "and to the point, highlighting those skills and experiences"
        "which are most relevant to the job posting ({job_posting_url})."
        "Tailor the cover letter to the specific role and company."
        "Express why you are interested in the role and how your"
        "experience aligns with the job requirements."
    ),
    context=[research_task, profile_task],
    agent=cover_letter_writer
)


In [130]:
# Task for Cover letter reviewer Agent: Review cover letter
review_cover_letter_task = Task(
    description=(
        "Review cover letters for job applications. "
        "Compare them to the provided job requirements at ({job_posting_url}) and give detailed feedback to the cover letter writer. "
        "Focus on how well the candidate's skills and experiences match the job requirements. "
        "Provide suggestions for improving the cover letter to better align with the job description. "
        "The feedback should be constructive and actionable, aimed at helping the cover letter writer refine their "
        "presentation and increase their chances of success."
    ),
    expected_output=(
        "A comprehensive feedback report on the cover letter, highlighting strengths, weaknesses, and "
        "specific suggestions for improvement. The report should include a clear assessment of how well "
        "the cover letter matches the job requirements and any additional advice for making the application stand out."
    ),
    context=[research_task, profile_task, cover_letter_compose_task],
    agent=cover_letter_reviewer,
    async_execution=False,
)

In [131]:
# Task for QA Agent: Check Consistency
check_consistency_task = Task(
    description=(
        "Ensure that the contents of the cover letter are factual and consistent with the job "
        "requirements {job_posting_url} and the candidate's resume {resume_path}. "
        "Cross-reference the details in each document to identify any discrepancies or gaps. "
        "Focus on ensuring that the information is aligned and presents a cohesive narrative. "
        "Highlight any inconsistencies found and suggest possible corrections for the cover letter writer agent."
    ),
    expected_output=(
        "A detailed consistency report that outlines any discrepancies or gaps found between the cover letter and "
        "other relevant documents (candidate's resume, candidate's linkedIn profile, the job requirements, etc). "
        "The report should provide suggestions for corrections to ensure "
        "the cover letter is aligned with job requirements and canditate documents, and presents a cohesive narrative."
    ),
    agent=qa_agent,
    context=[research_task, profile_task, cover_letter_compose_task],
    async_execution=False
)

## Creating the Crew

In [132]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

### Define your personal inputs here!

In [133]:
job_posting_url = "https://www.anytime-ai.com/careers/"
resume_path = "data/input/2024 (6).pdf"
linkedin_url = "https://www.linkedin.com/in/seth-donaldson/"

In [134]:
cover_letter_inputs = {
    'job_posting_url': job_posting_url,
    'resume_path': resume_path,
    'linkedin_url': linkedin_url,
}

In [135]:
# reassign the path of the output files for each of the tasks
profile_task.output_file = 'candidate_profile.txt'
research_task.output_file = 'job_requirements.txt'
cover_letter_compose_task.output_file = 'cover_letter.txt'
review_cover_letter_task.output_file = 'cover_letter_review.txt'
check_consistency_task.output_file = 'consistency_report.txt'

# Tool definitions
scrape_linkedin_tool = ScrapeWebsiteTool(website_url=linkedin_url)
scrape_job_posting_tool = ScrapeWebsiteTool(website_url=job_posting_url)
semantic_search_resume = PDFSearchTool(pdf=resume_path)
# semantic_search_resume = PDFSearchTool(pdf=resume_path)

metadata 2:  None
metadata 3:  None


In [136]:
profiler.tools = [scrape_linkedin_tool, semantic_search_resume]
job_researcher.tools = [scrape_job_posting_tool]
cover_letter_writer.tools = [scrape_linkedin_tool, scrape_job_posting_tool, semantic_search_resume]
cover_letter_reviewer.tools = [scrape_job_posting_tool]
qa_agent.tools = [scrape_linkedin_tool, scrape_job_posting_tool, semantic_search_resume]

In [137]:
cover_letter_crew = Crew(
    agents=[
            profiler,
            job_researcher,
            cover_letter_writer,
            cover_letter_reviewer,
            qa_agent
            ],

    tasks=[
            profile_task,
            research_task,
            cover_letter_compose_task,
            review_cover_letter_task,
            check_consistency_task
           ],
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    # process=Process.sequential,
    verbose=True,
#     memory=True,
#     cache=True,
    # full_output=True,
    # output_log_file='crew_log.txt'
)

2024-06-25 17:11:26,017 - 8318979072 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Run the Crew

In [138]:
### this execution will take a few minutes to run
result = cover_letter_crew.kickoff(inputs=cover_letter_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Compile a detailed and comprehensive personal and professional profile using the candidate's resume (data/input/2024 (6).pdf) and LinkedIn profile (https://www.linkedin.com/in/seth-donaldson/). Utilize tools to extract and synthesize information from these sources.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of compiling a detailed personal and professional profile using the candidate's resume and LinkedIn profile to the Personal Profiler for Job Candidates. They will be able to extract and synthesize information effectively.

Action:
Delegate work to co-worker

Action Input:
{
    "task": "Compile a detailed and comprehensive personal and professional profile using the candidate's resume and LinkedIn profile",
    "context": "I need you to extract all relevant skills, work experiences, project experiences, contributions, interests, and communication style from the candidate's resume and LinkedIn 

In [139]:
print(result)

After thorough cross-referencing of Seth Donaldson's resume, LinkedIn profile, and relevant experience data with the cover letter for the Machine Learning Engineer position at Anytime AI, I have identified a few discrepancies and gaps that need to be addressed.

1. The cover letter mentions Seth's experience in "deepfake technology" and "Mars rover development," but it does not specify his roles and achievements in these projects. It would be beneficial to provide more details on his leadership as a Team Lead and Computer Vision Engineer for the BYU Mars Rover project and his contributions to the deepfake technology project.

2. The cover letter highlights Seth's honors and awards, including the Microsoft Scholarship and 2nd place in a Student Research Conference, but it does not mention his GRE and ACT scores or language proficiency. Including these details would further showcase his academic achievements and diverse skill set.

3. The cover letter mentions Seth's passion for learning

- Dislplay the generated `cover_letter.md` file.

In [140]:
from IPython.display import Markdown, display
display(Markdown("candidate_profile.txt"))

Seth Donaldson

Passionate about learning languages. Very interested in deep learning. Experienced in…

Experience & Education
Etsy
- ******
- ******** ******** **
- ******** **********
- ******** ****** ******** *********
- ****** ******** ************
- ******** ****** ********
- ******** ********** ** *** **** ** *** ****
- ****** ** ******* - ** 
2021 - 2022

Projects
1. Phife Lives: "The Space Program" - A Tribe Called Quest Deepfake
   Nov 2019 - Jun 2020
   Used deep learning to insert the legendary MC, Malik "Phife Dawg" Taylor, into his own music video posthumously.

2. BYU Mars Rover | Team Lead | Computer Vision Engineer
   Aug 2019 - May 2020
   Managed a team of 30+ students to build a functional Mars rover for an annual competition, achieving a preliminary performance score of 90/100. Developed the autonomous navigation & computer vision of a Mars rover to detect targets and obstacles in a competition environment, improving target detection accuracy by 8% over the previous year’s model.

3. Data-Driven Discovery of Models (D3M)
   Dec 2017 - May 2019

4. Garmin Pilot: Weight and Balance
   May 2018 - Aug 2018

5. Is It Cold Outside?
   Dec 2017 - Jan 2018
   Developed an app to determine whether it is cold outside based on location and weather factors.

6. BYU Mobile App
   Jun 2016 - Nov 2017
   Worked on building the entire BYU Android app, contributing to features such as Mobile class registration, Y-Time, My Financial Center, Map My Classes, Parking, Cougar Cash, My Finals Schedule, My Next Class, Printers, AED Locator, Galleries, and Job Openings.

Honors & Awards
- Microsoft Scholarship, Microsoft, Jun 2018
- 2nd Place in Student Research Conference (Research Presentation), BYU College of Physical and Mathematical Sciences, Mar 2018
- Placed 2nd among 40+ other graduate and undergraduate students for the quality of research presentation focused on using machine learning to improve machine learning.
- BYU Dean's List, Marriott School of Management, Jan 2017

Test Scores
- GRE Quantitative Score: 164, Aug 2019
- GRE Verbal Reasoning Score: 161, Aug 2019
- ACT Score: 27, Oct 2012

Languages
- English
- Portuguese (Limited working proficiency)
- Japanese (Elementary proficiency)

In [141]:
display(Markdown("cover_letter.txt"))

Seth Donaldson
123 Main Street
Anytown, USA 12345
seth.donaldson@email.com
[Date]

Hiring Manager
Anytime AI
456 Tech Lane
Tech City, USA 54321

Dear Hiring Manager,

I am writing to express my interest in the Machine Learning Engineer position at Anytime AI. With a background in Full Stack Software Engineering and Machine Learning, along with a passion for learning languages and deep learning, I am excited about the opportunity to contribute to your innovative team.

I was particularly impressed by Seth Donaldson's experience working on projects such as deepfake technology, Mars rover development, and app development. His leadership as a Team Lead and Computer Vision Engineer for the BYU Mars Rover project showcases his ability to manage a team and achieve outstanding results. Additionally, his work on projects like "Phife Lives: The Space Program" and "Is It Cold Outside?" demonstrates his creativity and problem-solving skills.

Furthermore, Seth's honors and awards, including the Microsoft Scholarship and placing 2nd in a Student Research Conference, highlight his dedication to excellence and continuous learning. His strong test scores in GRE and ACT, as well as proficiency in multiple languages such as English, Portuguese, and Japanese, further showcase his diverse skill set and adaptability.

I believe that my background in machine learning, computer vision, and team leadership make me a strong candidate for this position. I am excited about the prospect of joining a dynamic team like yours and contributing to cutting-edge projects in the tech space.

Thank you for considering my application. I look forward to the opportunity to discuss how my skills and experiences align with the needs of Anytime AI. Please find my resume attached for your review.

Warm regards,

Seth Donaldson